# Гипотеза о `font_size`

Предполагается следующее: ширина текста пропорциональна `font_size`.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], ".."))
from util import util

In [2]:
def add_basic_width(df, basic_size):
    def calc_basic_width(r):
        basic_width = int(r[r.font_size == basic_size].iloc[0].width)
        r["basic_width"] = pd.Series(basic_width, index=r.index)
        return r

    df["basic_width"] = np.nan
    return df.groupby("char").apply(calc_basic_width)

In [3]:
BASIC_FONT_SIZE = 14
BASIC_FONT = util.Font("Lucida Grande", BASIC_FONT_SIZE, "normal")

In [4]:
char_widths_df = util.get_df("../data/full/char_widths.csv", "all")
char_widths_df = util.filter_by_font(char_widths_df, BASIC_FONT, filters=["family", "face"])
char_widths_df

,char_id,char,alphabet,font_size,width
0,65,A,basic_latin,9,7
1,66,B,basic_latin,9,8
2,67,C,basic_latin,9,9
3,68,D,basic_latin,9,9
4,69,E,basic_latin,9,8
...,...,...,...,...,...
1207,123,{,russian,20,9
1208,124,|,russian,20,6
1209,125,},russian,20,9
1210,126,~,russian,20,16


In [5]:
font_sizes = list(char_widths_df.font_size.unique())
font_sizes

[9, 11, 12, 14, 16, 20]

In [6]:
char_widths_df = add_basic_width(char_widths_df, basic_size=BASIC_FONT_SIZE)
char_widths_df["width_coeff"] = char_widths_df.width / char_widths_df.basic_width
char_widths_df["height_coeff"] = char_widths_df.font_size / BASIC_FONT_SIZE
char_widths_df["stretch_coeff"] = char_widths_df.width_coeff / char_widths_df.height_coeff
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff
0,65,A,basic_latin,9,7,13,0.538462,0.642857,0.837607
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265
2,67,C,basic_latin,9,9,14,0.642857,0.642857,1.000000
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000
4,69,E,basic_latin,9,8,13,0.615385,0.642857,0.957265


Пусть для символа `X` и размера шрифта `S` величина `w(X, S)` означает ширину символа в пикселях.

**Гипотеза A:**
Если величина `'stretch_coeff'` близка к константе `C`, то зная величину `w(X, B)` для базового размера `B`, величину для размера `S` можно вычислить по формуле `w(X, S) = C * (S / B) * w(X, B)`. Т.е. для пересчета модели от базового размера `B` к некоторому новому размеру `S`, достаточно знать одну заранее вычисленную константу.

**Гипотеза B:**
Если `'stretch_coeff'` не константна, но связана с `'basic_width'` некоторой формулой `C(w) = f(w)`, то можно для вычислений использовать формулу `w(X, S) = C(w(X, B)) * (S / B) * w(X, B)`. Очевидный минус в том, что для вычислений нужно знать не константу, а некоторую функцию (вернее, ее значения для различных `'basic_width'`).

## Проверка гипотезы A

In [7]:
ggplot(char_widths_df) + \
    geom_density(aes(x="stretch_coeff"), color="black", fill="#8da0cb") + \
    ggtitle("Вариация 'stretch_coeff'")

In [8]:
ggplot(char_widths_df) + \
    geom_boxplot(aes("alphabet", "stretch_coeff", fill="alphabet")) + \
    ggtitle("Диапазон изменения 'stretch_coeff' для каждого алфавита")

Очевидно, что `'stretch_coeff'` совсем не константа. Однако, диапазон изменения выглядит небольшим. Как понять, это много или мало для прогнозирования? Попробуем делать предсказания с помощью медианного значения `'stretch_coeff'`.

In [9]:
stretch_coeff = char_widths_df.stretch_coeff.median()
stretch_coeff

0.9898989898989898

In [10]:
char_widths_df["stretched_width"] = np.round(stretch_coeff * char_widths_df.font_size / BASIC_FONT_SIZE * char_widths_df.basic_width).astype(int)
char_widths_df["stretched_width_error"] = char_widths_df.stretched_width - char_widths_df.width
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff,stretched_width,stretched_width_error
0,65,A,basic_latin,9,7,13,0.538462,0.642857,0.837607,8,1
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0
2,67,C,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0
4,69,E,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0


In [11]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="stretched_width_error", fill="font_size"), color="black", \
             tooltips=layer_tooltips().format("@font_size", 'd').title("Font size: @font_size")\
                                      .format("@stretched_width_error", 'd').line("error|@stretched_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count..")\
                                      .line("alphabet|@alphabet")) + \
    scale_fill_brewer(type="seq", palette="Oranges", direction=-1, breaks=font_sizes) + \
    facet_grid(x="alphabet", y="font_size")

Из графиков видно, что:

- Ошибки встречаются. Иногда больше чем на пиксель. И они будут накапливаться с увеличением длины текста.

- Чаще идет ошибка в большую сторону, т.е. ширина будет скорее завышаться.

- Когда шрифт больше базового, ошибок больше, чем когда он меньше. Возможно, это означает, что все-таки правильнее брать в качестве базового шрифт бóльшего размера.

## Проверка гипотезы B

In [12]:
ggplot(char_widths_df, aes("basic_width", "stretch_coeff")) + \
    geom_boxplot() + \
    ggtitle("Изменение 'stretch_coeff' в зависимости от базовой ширины символов") + \
    ggsize(800, 400) + \
    theme_minimal()

In [13]:
f_df = char_widths_df.groupby("basic_width").stretch_coeff.median()
ggplot(f_df.reset_index()) + \
    geom_line(aes("basic_width", "stretch_coeff")) + \
    ggtitle("f(x)")

In [14]:
char_widths_df["fstretched_width"] = np.round(char_widths_df.basic_width.replace(f_df.to_dict()) * char_widths_df.font_size / BASIC_FONT_SIZE * char_widths_df.basic_width).astype(int)
char_widths_df["fstretched_width_error"] = char_widths_df.fstretched_width - char_widths_df.width
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff,stretched_width,stretched_width_error,fstretched_width,fstretched_width_error
0,65,A,basic_latin,9,7,13,0.538462,0.642857,0.837607,8,1,8,1
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0,8,0
2,67,C,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0,9,0
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0,9,0
4,69,E,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0,8,0


In [15]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="fstretched_width_error", fill="font_size"), color="black", \
             tooltips=layer_tooltips().format("@font_size", 'd').title("Font size: @font_size")\
                                      .format("@fstretched_width_error", 'd').line("error|@fstretched_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count..")\
                                      .line("alphabet|@alphabet")) + \
    scale_fill_brewer(type="seq", palette="Oranges", direction=-1, breaks=font_sizes) + \
    facet_grid(x="alphabet", y="font_size")

Из графиков видно, что:

- Ошибки все так же встречаются.

- Ошибок меньше, чем в предыдущей модели, но они распределены менее нормально - что приведет к большему перекосу на более длинных текстах.

- Когда шрифт больше базового, модель ведет себя заметно лучше, чем предыдущая. Раза в полтора-два.

- При этом, такая модель допускает ошибки в случае, когда надо предсказать базовый шрифт - чего быть вообще не должно!

## Изучение второй модели

In [16]:
pf_df = char_widths_df.groupby(["font_size", "basic_width"]).stretch_coeff.median()
ggplot(pf_df.reset_index()) + \
    geom_line(aes("basic_width", "stretch_coeff")) + \
    facet_grid(y="font_size")

Ну конечно, ошибок предсказания было бы меньше, и модель вела бы себя лучше, если бы все эти функции были растяжением по вертикали (для разных коэффициентов) одной и той же функции! Но, очевидно, это не так.

Если для каждого размера шрфита поведение растяжения выглядит по-своему, то можно уже не учитывать коэффициент `S / B` в формулах - он все равно будет перекрыт коэффициентами более сложной модели (если придется использовать различные модели для различных размеров).

In [17]:
ggplot(char_widths_df) + \
    geom_point(aes("basic_width", "width")) + \
    geom_smooth(aes("basic_width", "width")) + \
    facet_wrap(facets="font_size") + \
    coord_fixed() + \
    ggsize(800, 800)

Из последних графиков создается впечатление, что может сработать формула `w(X, S) = a(S) * w(X, B) + b(S)`. Это будет **гипотеза C**.

## Проверка гипотезы C

In [18]:
def append_ab(df, ab_names={"a": "a", "b": "b"}):
    X = df[["basic_width"]]
    y = df[["width"]]
    model = LinearRegression().fit(X, y)
    df[ab_names["a"]] = np.tile(model.coef_.reshape(-1), df.shape[0])
    df[ab_names["b"]] = np.tile(model.intercept_, df.shape[0])
    return df

In [19]:
char_widths_df = pd.concat([
    append_ab(char_widths_df[char_widths_df.font_size == font_size].copy(), {"a": "a_coeff", "b": "b_coeff"})
    for font_size in font_sizes
])
char_widths_df["predicted_width"] = np.round(char_widths_df.a_coeff * char_widths_df.basic_width + char_widths_df.b_coeff).astype(int)
char_widths_df["predicted_width_error"] = char_widths_df.predicted_width - char_widths_df.width
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff,stretched_width,stretched_width_error,fstretched_width,fstretched_width_error,a_coeff,b_coeff,predicted_width,predicted_width_error
0,65,A,basic_latin,9,7,13,0.538462,0.642857,0.837607,8,1,8,1,0.630023,0.009811,8,1
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0,8,0,0.630023,0.009811,8,0
2,67,C,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0,9,0,0.630023,0.009811,9,0
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0,9,0,0.630023,0.009811,9,0
4,69,E,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0,8,0,0.630023,0.009811,8,0


In [20]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="predicted_width_error", fill="font_size"), color="black", \
             tooltips=layer_tooltips().format("@font_size", 'd').title("Font size: @font_size")\
                                      .format("@predicted_width_error", 'd').line("error|@predicted_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count..")\
                                      .line("alphabet|@alphabet")) + \
    scale_fill_brewer(type="seq", palette="Oranges", direction=-1, breaks=font_sizes) + \
    facet_grid(x="alphabet", y="font_size")

Кажется, что последняя модель, хоть ненамного, но лучше. Ошибок стало меньше, и их распределение чуть больше похоже на нормальное. Возможно, небольшое улучшение можно получить, заменив линейную модель на полиномиальную.

In [21]:
coeff_df = char_widths_df[["font_size", "a_coeff", "b_coeff"]].drop_duplicates()
coeff_df

,font_size,a_coeff,b_coeff
0,9,0.630023,9.811302e-03
202,11,0.766398,3.467109e-02
404,12,0.838975,-1.722231e-01
606,14,1.000000,-1.776357e-15
808,16,1.054812,4.250177e-01
1010,20,1.360745,6.365371e-01


In [22]:
ggplot(coeff_df) + \
    geom_line(aes("font_size", "a_coeff")) + \
    ggtitle("Зависимость коэффициента 'a' от размера шрифта")

In [23]:
ggplot(coeff_df) + \
    geom_line(aes("font_size", "b_coeff")) + \
    ggtitle("Зависимость коэффициента 'b' от размера шрифта")